HW4
1g

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


df = pd.read_csv('Final_Imputed_Whole.csv',index_col=0)
df_train = df.iloc[0:1495,:]
df_test = df.iloc[1495:1994,:]
df_test.reset_index(drop=True,inplace=True)
X = df_train.drop(columns=['ViolentCrimesPerPop'])
X_final_test = df_test.drop(columns=['ViolentCrimesPerPop'])
y = df_train['ViolentCrimesPerPop']
y_final_test = df_test['ViolentCrimesPerPop']
print(X)
print(y)
print(X_final_test)
print(y_final_test)

The value of lambda or alpha will be selected from the following set of values:
{5*10^-3,5*10^-2,5*10^-1,5*10^0,5*10^1}
Thus we'll select lambda/alpha that corresponds to the least CV error

We'll use 5-fold cross validation on the training set, compute average MSE over these 5-folds for each lambda, select lambda giving least MSE

Once lambda selected, we'll refit model on the entire training data using that lambda, obtain MSE over test finally

In [ ]:
alpha = np.array([5*(10**(-3)),5*(10**(-2)),5*(10**(-1)),5*(10**(0)),5*(10**(1))])
alpha_pointer = np.arange(0,5)
alpha_mse_sum = np.zeros(5)
kf = KFold(n_splits=5,shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    i=0
    for i in alpha_pointer:
        ridge_reg = Ridge(alpha=alpha[i],fit_intercept=True,normalize=False) #Already Normalized, Fit Intercept
        ridge_fit = ridge_reg.fit(X_train,y_train)
        y_predicted = ridge_fit.predict(X_test)
        mse_interim = mean_squared_error(y_test,y_predicted)
        alpha_mse_sum[i] = alpha_mse_sum[i] + mse_interim #alpha_mse_sum stores mse in indices corresponding to alphas, and 
                                                          #it keeps summing mse's for each alpha over every single fold, 
                                                          #final mse obtained by dividing by 5 
                
final_alpha_mse = alpha_mse_sum/5 #alpha mse were summed, dividing by 5 gives us the average mse for each alpha over 5-folds


selected_alpha_index = np.argmin(final_alpha_mse) #returns index with minimal mse, corresponding to index for alpha

print('The alpha selected, via 5-fold CV is : \n',alpha[selected_alpha_index])

final_ridge = Ridge(alpha=alpha[selected_alpha_index],fit_intercept=True,normalize=False)

final_fit = final_ridge.fit(X,y) #fitting on the entire training set

y_final_predicted = final_fit.predict(X_final_test)

print('The final y predicted over test set with the best selected alpha/lambda is : \n',y_final_predicted)
print('The y_test for comparison is : \n',y_final_test)

final_mse = mean_squared_error(y_final_test,y_final_predicted)

print('The corresponding MSE is : \n',final_mse)
        

Thus test MSE for Ridge Regression is 0.017
Test MSE for Linear Regression was 0.056

Thus Ridge Regression, with lambda = 5 performs better than Linear Regression